Imports and installs

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/thomas-allard/APPM5720/blob/master/NavierStokeExample.ipynb)


In [1]:
!pip install sciann
!pip install matplotlib
!pip install scipy
#!/usr/bin/env python3


## exact value of the parameters: lambda1 = 0.9967, lambda2 = 0.0110
import numpy as np 
import sciann as sn 
import matplotlib.pyplot as plt
import scipy.io

import os
# sns.set_theme()

     |████████████████████████████████| 173 kB 7.1 MB/s 
     |████████████████████████████████| 561 kB 40.7 MB/s 
     |████████████████████████████████| 460.3 MB 7.8 kB/s 
     |████████████████████████████████| 4.0 MB 36.2 MB/s 
     |████████████████████████████████| 132 kB 56.9 MB/s 
     |████████████████████████████████| 1.2 MB 47.9 MB/s 
     |████████████████████████████████| 14.8 MB 42.5 MB/s 
     |████████████████████████████████| 462 kB 73.0 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68723 sha256=d64dac8d72751b7cd2bbeaeba3aeac833a1aa2152d5dc836a7f1efa684082d4a
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: numpy
    Foun

---------------------- SCIANN 0.6.5.1 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://www.sciencedirect.com/science/article/pii/S0045782520307374", 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 

 Need support or would like to contribute, please join sciann`s slack group: 
 +  "https://join.slack.com/t/sciann/shared_invite/zt-ne1f5jlx-k_dY8RGo3ZreDXwz0f~CeA" 
 
TensorFlow Version: 2.5.3 
Python Version: 3.7.12 (default, Jan 15 2022, 18:48:18) 
[GCC 7.5.0] 



In [3]:
# define function for subsampling data
def PrepareData(num_data, random=True):
    
    # Get data file from: 
    #         https://github.com/maziarraissi/PINNs/tree/master/main/Data/cylinder_nektar_wake.mat
    data = scipy.io.loadmat('cylinder_nektar_wake.mat')
    
    U_star = data['U_star'] # N x 2 x T
    P_star = data['p_star'] # N x T
    t_star = data['t'] # T x 1
    X_star = data['X_star'] # N x 2
    
    N = X_star.shape[0]
    T = t_star.shape[0]
    
    # Rearrange Data 
    XX = np.tile(X_star[:,0:1], (1,T)) # N x T
    YY = np.tile(X_star[:,1:2], (1,T)) # N x T
    TT = np.tile(t_star, (1,N)).T # N x T
    
    UU = U_star[:,0,:] # N x T
    VV = U_star[:,1,:] # N x T
    PP = P_star # N x T
    
    # Pick random data.
    if random:
        idx = np.random.choice(N*T, num_data, replace=False)
    else:
        idx = np.arange(0, N*T)
    
    x = XX.flatten()[idx,None] # NT x 1
    y = YY.flatten()[idx,None] # NT x 1
    t = TT.flatten()[idx,None] # NT x 1
    
    u = UU.flatten()[idx,None] # NT x 1
    v = VV.flatten()[idx,None] # NT x 1
    p = PP.flatten()[idx,None] # NT x 1
 
    return (x,y,t,u,v,p)


In [4]:
# Subsample data for training
x_train, y_train, t_train, u_train, v_train, p_train = PrepareData(5000, random=True)

In [5]:
# Setting up cost function
layers = 10
neurons = 10

x = sn.Variable("x", dtype='float64')
y = sn.Variable("y", dtype='float64')
t = sn.Variable("t", dtype='float64')

P = sn.Functional("P", [x, y, t], layers*[neurons], 'tanh')
Psi = sn.Functional("Psi", [x, y, t], layers*[neurons], 'tanh')

lambda1 = sn.Parameter(np.random.rand(), inputs=[x,y,t], name="lambda1")
lambda2 = sn.Parameter(np.random.rand(), inputs=[x,y,t], name="lambda2")
## true value lamdba1 = 

u = sn.diff(Psi, y)
v = -sn.diff(Psi, x)

u_t = sn.diff(u, t)
u_x = sn.diff(u, x)
u_y = sn.diff(u, y)
u_xx = sn.diff(u, x, order=2)
u_yy = sn.diff(u, y, order=2)

v_t = sn.diff(v, t)
v_x = sn.diff(v, x)
v_y = sn.diff(v, y)
v_xx = sn.diff(v, x, order=2)
v_yy = sn.diff(v, y, order=2)

p_x = sn.diff(P, x)
p_y = sn.diff(P, y)

# Define constraints 
d1 = sn.Data(u)
d2 = sn.Data(v)
d3 = sn.Data(P)

c1 = sn.Tie(-p_x, u_t+lambda1*(u*u_x+v*u_y)-lambda2*(u_xx+u_yy))
c2 = sn.Tie(-p_y, v_t+lambda1*(u*v_x+v*v_y)-lambda2*(v_xx+v_yy))
c3 = sn.Data(u_x + v_y)

c4 = Psi*0.0

# Define the optimization model (set of inputs and constraints)
model = sn.SciModel(
    inputs=[x, y, t],
    targets=[d1, d2, d3, c1, c2, c3, c4],
    loss_func="mse",
    plot_to_file='NS-Model.png'
)

input_data = [x_train, y_train, t_train]

data_d1 = u_train
data_d2 = v_train
data_d3 = p_train
data_c1 = 'zeros'
data_c2 = 'zeros'
data_c3 = 'zeros'
data_c4 = 'zeros'
target_data = [data_d1, data_d2, data_d3, data_c1, data_c2, data_c3, data_c4]

In [6]:
## hyper parameter define

Epochs= 5000
Batch_size=128
Learning_rate=0.001
Reduce_lr_after=1300

#log the data

    
##train PINN

history = model.train(
    x_true=input_data,
    y_true=target_data,
    epochs= Epochs,
    batch_size= Batch_size,
    shuffle=True,
    learning_rate= Learning_rate,
    reduce_lr_after= Reduce_lr_after,
    stop_loss_value=1e-8,
    verbose=1,
    log_parameters={'parameters':[lambda1,lambda2],
                    'freq':1}
)



Total samples: 5000 
Batch size: 128 
Total batches: 40 

Epoch 1/5000


KeyboardInterrupt: ignored

Plots and things!

In [ ]:

## logs and plots
model.save_weights('trained-navier-stokes.hdf5')
## print iterated parameters value
print("lambda1: {},  lambda2: {}".format(lambda1.value, lambda2.value))
with open(f'../logs/log_layers{layers}_neurons{neurons}_epochs{Epochs}_batchsize{Batch_size}_learning_rate{Learning_rate}_reduce_lr_after{Reduce_lr_after}/parameters_{layers}_{neurons}.txt', 'w') as f:
    f.write(f"lambda1: {lambda1.value},  lambda2: {lambda2.value}")
print("lambda1: {},  lambda2: {}".format(lambda1.value, lambda2.value))

## print loss function value
print("loss: {}".format(history.history['loss']))
with open(f'../logs/log_layers{layers}_neurons{neurons}_epochs{Epochs}_batchsize{Batch_size}_learning_rate{Learning_rate}_reduce_lr_after{Reduce_lr_after}/loss_{layers}_{neurons}.txt', 'w') as f:
    f.write(f"loss: {history.history['loss']}")
## plot loss function
plt.figure(f'Train_Loss_{layers}_{neurons}')
plt.semilogy(history.history['loss'])
plt.xlabel('epochs')
plt.ylabel('loss')
plt.savefig(f'../logs/log_layers{layers}_neurons{neurons}_epochs{Epochs}_batchsize{Batch_size}_learning_rate{Learning_rate}_reduce_lr_after{Reduce_lr_after}/loss_{layers}_{neurons}.png')
plt.semilogy(history.history['loss'])
plt.xlabel('epochs')
plt.ylabel('loss')


What should classmates do with trained PINNs?
- Plot pressure distribution at some time
- Print lambdas